In [1]:
import pandas as pd
import numpy as np

**Una Serie con Índices Múltiples**

* Comencemos considerando cómo podríamos representar datos bidimensionales dentro de una Serie unidimensional. Para ser concretos, consideraremos una serie de datos donde cada punto tiene una clave característica y numérica.

**La manera incorrecta**
* Supongamos que deseas seguir datos sobre estados de dos años diferentes. Usando las herramientas de Pandas que ya hemos cubierto, podrías sentir la tentación de simplemente usar tuplas de Python como claves:

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)
         ]

populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561
               ]

pop = pd.Series(populations,index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

**Ejercicio Propio**

In [3]:
from itertools import groupby
from operator import itemgetter

# Paso 1: Definir los datos
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)
         ]

populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561
               ]

# Paso 2: Crear una lista de tuplas (estado, año, población)
data = list(zip(index, populations))
print(data)
# Paso 3: Ordenar la lista de tuplas por año usando lambda
data_sorted = sorted(data,key=lambda x: x[0][1])

# grouped_data = groupby(data_sorted,key=lambda x: x[0][1])
grouped_data = {key: list (group) for key, group in groupby(data_sorted,key=lambda x: x[0][1])}

for year, group in grouped_data.items():
    print(f"Año: {year}")
    for (state,_),population in group:
        print(f"  Estado: {state}, Población: {population}")        

[(('California', 2000), 33871648), (('California', 2010), 37253956), (('New York', 2000), 18976457), (('New York', 2010), 19378102), (('Texas', 2000), 20851820), (('Texas', 2010), 25145561)]
Año: 2000
  Estado: California, Población: 33871648
  Estado: New York, Población: 18976457
  Estado: Texas, Población: 20851820
Año: 2010
  Estado: California, Población: 37253956
  Estado: New York, Población: 19378102
  Estado: Texas, Población: 25145561


Con este esquema de indexación, puedes indexar o dividir directamente la serie basándote en este índice múltiple:

In [4]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [5]:
pop.index

Index([('California', 2000), ('California', 2010),   ('New York', 2000),
         ('New York', 2010),      ('Texas', 2000),      ('Texas', 2010)],
      dtype='object')

"Pero la conveniencia termina ahí. Por ejemplo, si necesitas seleccionar todos los valores desde 2010, necesitarás realizar cierta manipulación engorrosa (y potencialmente lenta) para lograrlo:"

In [6]:
data2010 = pop[[i for i in pop.index if[1] == 2010]]
data2010

Series([], dtype: int64)

Esto produce el resultado deseado, pero no es tan limpio (ni tan eficiente para conjuntos de datos grandes) como la sintaxis de rebanado que hemos llegado a apreciar en Pandas.

**La mejor manera: Pandas MultiIndex**

* Afortunadamente, Pandas ofrece una mejor solución. Nuestro índice basado en tuplas es básicamente un índice múltiple rudimentario, y el tipo MultiIndex de Pandas nos proporciona el tipo de operaciones que deseamos tener. Podemos crear un MultiIndex a partir de las tuplas de la siguiente manera:

In [7]:
index

[('California', 2000),
 ('California', 2010),
 ('New York', 2000),
 ('New York', 2010),
 ('Texas', 2000),
 ('Texas', 2010)]

In [8]:
index = pd.MultiIndex.from_tuples(index)
print(index)

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )


Observa que el MultiIndex contiene múltiples niveles de indexación; en este caso, los nombres de los estados y los años, así como múltiples etiquetas para cada punto de datos que codifican estos niveles.
Si volvemos a indexar nuestra serie con este MultiIndex, veremos la representación jerárquica de los datos:

In [9]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Aquí, las dos primeras columnas de la representación de la Serie muestran los valores de los índices múltiples, mientras que la tercera columna muestra los datos. Observa que algunas entradas están ausentes en la primera columna: en esta representación de MultiIndex, cualquier entrada en blanco indica el mismo valor que la línea anterior.

Ahora, para acceder a todos los datos para los cuales el segundo índice es 2010, simplemente podemos usar la notación de rebanado de Pandas:

In [10]:
pop[:,2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

El resultado es un arreglo indexado de forma simple con las claves que nos interesan. Esta sintaxis es mucho más conveniente (¡y la operación es mucho más eficiente!) que la solución casera de indexación multi-index basada en tuplas con la que comenzamos. Ahora vamos a discutir más a fondo este tipo de operación de indexación en datos indexados jerárquicamente.

**Importante**

Las funciones stack y unstack en pandas se utilizan para reorganizar los datos en un DataFrame o Series con un MultiIndex. 

**Diferencias Clave**

* **stack:** Transforma columnas en niveles de índice. Utiliza df.stack() para "apilar" las columnas en el índice.

* **unstack:** Transforma niveles de índice en columnas. Utiliza df.unstack() para "desapilar" los niveles del índice en columnas.

**MultiIndex como una dimensión adicional**

* Puede que notes algo más aquí: fácilmente podríamos haber almacenado los mismos datos utilizando un DataFrame simple con etiquetas de índice y columnas. De hecho, Pandas está diseñado teniendo esta equivalencia en mente. 

* **El método unstack()** convertirá rápidamente una Serie indexada de forma múltiple en un DataFrame indexado convencionalmente.

In [11]:
pop_df = pop.unstack()
print(type(pop))
pop_df

<class 'pandas.core.series.Series'>


,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


**Naturalmente, el método stack() proporciona la operación opuesta:**

In [12]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

* Viendo esto, podrías preguntarte por qué nos molestaríamos con la indexación jerárquica en absoluto. La razón es simple: al igual que pudimos usar la indexación múltiple para representar datos bidimensionales dentro de una Serie unidimensional, también podemos usarla para representar datos de tres o más dimensiones en una Serie o DataFrame. 

* Cada nivel adicional en un MultiIndex representa una dimensión adicional de datos; aprovechar esta propiedad nos da mucha más flexibilidad en los tipos de datos que podemos representar. 

* Concretamente, podríamos querer agregar otra columna de datos demográficos para cada estado en cada año (por ejemplo, población menor de 18 años); con un MultiIndex, esto es tan fácil como agregar otra columna al DataFrame.

In [13]:
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [14]:
pop_df = pd.DataFrame(
    {
        'total' : pop,
        'under18' :  [9267089, 9284094,
                      4687374, 4318033,
                      5906301, 6879014]
    }
    )

pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [15]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18

California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [16]:
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


**Métodos de Creación de MultiIndex**

* La forma más directa de construir una Serie o DataFrame con índices múltiples es simplemente pasar una lista de dos o más arreglos de índices al constructor. Por ejemplo:

In [20]:
df = pd.DataFrame(np.random.rand(4,2),
                  index=[['a','a','b','b'],[1,2,1,2]],
                  columns=['data1','data2'])

df

data1     data2
a 1  0.977803  0.114306
  2  0.958589  0.948693
b 1  0.204562  0.118831
  2  0.391741  0.696237

**Constructores Explícitos de MultiIndex**

* Para obtener más flexibilidad en la construcción del índice, puedes utilizar en su lugar los constructores de métodos de clase disponibles en pd.MultiIndex. Por ejemplo, como hicimos antes, puedes construir el MultiIndex a partir de una simple lista de arreglos, proporcionando los valores de índice dentro de cada nivel.

In [24]:
arrays = [['a', 'a', 'b', 'b'], [1, 2, 1, 2]]
multi_index = pd.MultiIndex.from_arrays(arrays)
multi_index

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [25]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [26]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

**MultiIndex para columnas**

* En un DataFrame, las filas y las columnas son completamente simétricas, y al igual que las filas pueden tener varios niveles de índices, las columnas también pueden tener múltiples niveles. Considera lo siguiente, que es una simulación de algunos datos médicos (algo realistas):

In [28]:
index = pd.MultiIndex.from_product([[2013,2014],[1,2]],
                                   names=['year','visit'])

columns = pd.MultiIndex.from_product([['bob','Guido','Sue'],['hr','temp']],
                                     names=['subject','type'])

In [35]:
#mock some data
data = np.round(np.random.rand(4,6),1)
data[:,::2] *= 10
data += 37
data

array([[44. , 37.1, 44. , 37.5, 39. , 37.7],
       [44. , 37.6, 47. , 37.6, 38. , 37.6],
       [40. , 37.5, 44. , 37.7, 45. , 37.2],
       [46. , 37.2, 45. , 37.1, 44. , 37.6]])

In [36]:
#create the dataframe 

health_data = pd.DataFrame(data,index=index,columns=columns)
health_data

subject      bob       Guido         Sue      
type          hr  temp    hr  temp    hr  temp
year visit                                    
2013 1      44.0  37.1  44.0  37.5  39.0  37.7
     2      44.0  37.6  47.0  37.6  38.0  37.6
2014 1      40.0  37.5  44.0  37.7  45.0  37.2
     2      46.0  37.2  45.0  37.1  44.0  37.6

Aquí es donde podemos ver lo útil que puede ser el índice múltiple tanto para filas como para columnas. Estos datos son fundamentalmente de cuatro dimensiones, donde las dimensiones son el sujeto, el tipo de medición, el año y el número de visita. Con esto en su lugar, podemos, por ejemplo, indexar la columna de nivel superior por el nombre de la persona y obtener un DataFrame completo que contenga solo la información de esa persona:

In [37]:
health_data['Guido']

type          hr  temp
year visit            
2013 1      44.0  37.5
     2      47.0  37.6
2014 1      44.0  37.7
     2      45.0  37.1

pag 152